# Exemples d'implémentation d'algorithmes avec dask

## Nombres premiers

Voici un exemple d'implémentation d'un algorithme pour trouver les nombres premiers jusqu'à un certain nombre `n` en utilisant Dask.

In [21]:
from dask.distributed import Client
import time

client = Client()

def is_prime(x):
    for i in range(2, x // 2):
      if x % i == 0:
        return None
    return x

def launch(n):
  print(f"running for {n}")
  input = range(3, n)
  start = time.perf_counter()
  results = [x for x in input if is_prime(x) != None]
  end = time.perf_counter()
  print(f"Single threaded time: {end - start:.6f} seconds")

  start = time.perf_counter()
  futures = client.map(is_prime, input)
  results = client.gather(futures)
  end = time.perf_counter()
  print(f"Dask time: {end - start:.6f} seconds")

launch(100)
launch(10000)

/Users/yassinebenabbas/dev/github/yostane/lectures/docs/distrib/dask/.venv/lib/python3.14/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57719 instead
  warnings.warn(


running for 100
Single threaded time: 0.000035 seconds
Dask time: 0.074387 seconds
running for 10000
Single threaded time: 0.106620 seconds
Dask time: 5.429720 seconds


In [18]:
client.dashboard_link

'http://127.0.0.1:56011/status'

In [ ]:
client.close()